<a href="https://colab.research.google.com/github/alerodriguessf/Data_Preparation_Retail_Sales_Analysis_20241120/blob/main/Data_Preparation_Retail_Sales_Analysis_20241120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Project: Retail Sales Analysis
Author: Alexandre
Description:
    This script performs data cleaning, analysis, and visualization
    on a retail sales dataset. Key steps include:
    - Data inspection and cleaning
    - Handling missing values and outliers
    - Data type corrections
    - Generating visualizations for key metrics
    - Computing RFM (Recency, Frequency, Monetary) analysis
"""

# Required Libraries
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import sidetable
import missingno as msno
from ipywidgets import interact, widgets
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure required packages are installed
!pip install ydata-profiling --quiet
!pip install --upgrade numba --quiet
!pip install sidetable --quiet

# Step 1: Upload the dataset
from google.colab import files
uploaded = files.upload()

# Load the dataset
df = pd.read_csv('data.csv', encoding='latin-1')

# Data Inspection
print("Dataset Info:")
df.info()
print("\nDataset Description:")
print(df.describe())

# Generate an initial profile report
df.profile_report()

# Step 2: Handle Missing Values
# Check for missing values and calculate their percentage
df_missing = (
    df.isna().sum()
    .to_frame('missing_count')
    .join((df.isna().sum() / df.shape[0]).to_frame('missing_percentage'))
    .sort_values('missing_count', ascending=False)
)
print("\nMissing Values:")
print(df_missing)

# Drop rows with missing values in critical columns
df = df.dropna(subset=['Description', 'CustomerID'])

# Step 3: Filter Invalid Data
# Remove records with invalid unit prices or quantities
df = df[(df['UnitPrice'] > 0) & (df['Quantity'] > 0)]

# Step 4: Remove Duplicates
# Drop duplicate rows
df = df.drop_duplicates()

# Step 5: Correct Data Types
# Convert date column to datetime and customer ID to integer
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerID'] = df['CustomerID'].astype(int)

# Step 6: Handle Outliers
# Remove extreme outliers based on predefined thresholds
df = df[(df['Quantity'] <= 10000) & (df['UnitPrice'] <= 5000)]

# Step 7: Add New Columns
# Add a column for total price
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Step 8: Determine the Last Purchase Date
last_purchase_date = df['InvoiceDate'].max()
print(f"\nLast Purchase Date: {last_purchase_date}")

# Step 9: Generate Visualizations
# Top 10 countries by total sales
top_countries = df.groupby('Country')['TotalPrice'].sum().nlargest(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_countries.values, y=top_countries.index, palette='viridis')
plt.title('Top 10 Countries by Total Sales')
plt.xlabel('Total Sales')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

# Top 10 products by quantity sold
top_products = df.groupby('Description')['Quantity'].sum().nlargest(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_products.values, y=top_products.index, palette='plasma')
plt.title('Top 10 Products by Quantity Sold')
plt.xlabel('Quantity Sold')
plt.ylabel('Product')
plt.tight_layout()
plt.show()

# Total sales by month
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')
monthly_sales = df.groupby('YearMonth')['TotalPrice'].sum()
plt.figure(figsize=(12, 6))
monthly_sales.plot(kind='bar', color='skyblue')
plt.title('Total Sales by Month')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Step 10: RFM Analysis
# Compute Recency, Frequency, and Monetary values
rfm = df.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (last_purchase_date - x.max()).days),
    Frequency=('InvoiceNo', 'nunique'),
    Monetary=('TotalPrice', 'sum')
)
print("\nRFM Analysis:")
print(rfm.head())
